# Dataset Index Generation
Generates indices for train, validation and test sets

In [1]:
import h5py
from progressbar import *
import re
import numpy as np

## Options

In [2]:
n_test_files = 400
n_val_files = 100
labels = (0, 1)

## Load dataset

In [3]:
#data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_1p5ns.h5"
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_2p5ns.h5"
f = h5py.File(data_path, "r")
event_labels = np.array(f['labels'])
root_files = np.array(f['root_files']).astype(str)

## Find the files of each label and indices of each file

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [5]:
files_in_labels = {l: sorted(set(root_files[event_labels==l]), key=natural_keys) for l in labels}
idxs_in_files = {f: range(i, i+c) for f,i,c in zip(*np.unique(root_files, return_index=True, return_counts=True))}

In [6]:
for l, f in files_in_labels.items():
    print("label", l,"has", len(f),"files and ", sum([len(idxs_in_files[i]) for i in f]), "indices")

label 0 has 3000 files and  8860171 indices
label 1 has 3000 files and  8821709 indices


## Create the splits

In [7]:
split_files = {"test_idxs":  [f for l in labels for f in files_in_labels[l][:n_test_files]],
               "val_idxs":   [f for l in labels for f in files_in_labels[l][n_test_files:n_test_files+n_val_files]],
               "train_idxs": [f for l in labels for f in files_in_labels[l][n_test_files+n_val_files:]]}
split_idxs = {k: [i for f in v for i in idxs_in_files[f]] for k, v in split_files.items()}

In [8]:
for s in split_files.keys():
    print(s,"has", len(split_files[s]),"files and", len(split_idxs[s]),"indices")

test_idxs has 800 files and 2357653 indices
val_idxs has 200 files and 589267 indices
train_idxs has 5000 files and 14734960 indices


In [9]:
# Verify that all events are uniquely accounted for
all_indices = np.concatenate(list(split_idxs.values()))
print(len(event_labels))
print(len(all_indices))
print(len(set(all_indices)))

17681880
17681880
17681880


## Save file

In [10]:
#np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_1p5ns_idxs.npz', **split_idxs)
np.savez('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short/tts_egamma/IWCD_mPMT_Short_eg_2p5ns_idxs.npz', **split_idxs)